In [ ]:
import MeerKAT_Mosaic_Calculator_base as msc

# MeerKAT Mosaic Calculator
Use the cell below to create a cube representing the weights or the sensitivity of a mosaic (in dependence of frequency, hence three dimensions). Notice that at this stage the single rms is either entered by hand or the HI line sensitivity can be calculated providing some parameters, but it is not considered to vary with frequency (only across the primary beam).

The peak rms for a single primary beam is entered with the parameter ``rmssingle`` (in Jy). If the parameter ``rmssingle`` is entered as the string "calculate", the script calculates the rms from the parameters ``t_int`` (integration time in hours), ``Nant`` (number of antennas), ``HI_velocity_width`` (HI velocity window over which to calculate the sensitivity) in km/s, ``obs_freq`` (observation frequency in Hz), ``f_tap`` (the weighting factor or penalty, a simple factor to increase the thermal rms by to account for the effects of using weighting schemes differing from natural weighting).

The user determines the size of the cube by providing the upper and lower boundaries. These are determined by the variables ``RAmin`` (minimum right ascension of cube in deg), ``RAmax`` (maximum right ascension of cube in deg), ``Demin`` (minimum declination of cube in deg), ``Demax`` (maximum declination of cube in deg), ``numin`` (minimum frequency of cube in Hz), ``numax`` (maximum frequency of cube in Hz). The voxel size is determined by the spatial voxel size ``cdeltxy`` (in deg) and the frequency voxel size ``cdeltnu`` in Hz.

The positions of the pointings are provided in a list ``ras`` of right ascensions and a list ``decs`` of declinations, either as floats in units of deg or in hexagesimal units (provided as strings). Optionally, the user supplies names for the pointings in a list ``names`` of strings. 

Alternatively, the user can automatically generate a grid confined by a polygon, if she sets ``grid_from_polygon = True``
The vertices are given by ``polypointsra`` and ``polypointsdec``, which are lists of numpy-readable angles (e.g. ``15.0``, ``'15deg'``, ``'1h00m00s'``, ``'15d00m00s'`` all for 15 deg). The regular polygonal grid, either 'hexagonal' or 'quadradic' (determined by parameter gridtype), has a separation of ``sep`` for the grid points (``sep`` in units of degrees), and is tilted by the angle tilt (in degrees). The lateral shift of the grid is determined by the position given by the pair ``startpoint``, and is by default the barycentre of the grid, if ``startpoint = None``. Startpoint should be chosen to lie inside the polygon. The parameter ``border`` (in degrees) determines a border of the grid towards the polygon. If ``border`` is positive, no grid point can have a distance lower than ``border`` towards the input polygon. If ``border`` is negative, grid points outside the polygon are allowed if their distance to a side of the polygon is less than the absolute value of ``border``. The pointings are automatically named ``prefix``-number using the parameter ``prefix`` as a prefix and increasing integers starting with 00.

The parameter ``pbshape`` determines the shape of the primary beam ``'Tapered Cos Squared'`` or ``'Mauchian'`` (see Mauch, et al. 2020, ApJ, 888, 61) or ``Gaussian``. The script will first calculate a weight map (or rather a cube with frequency as the third axis), the inverse of the noise map in units of the inverse of ``rmsrel``. If ``rmsrel`` is chosen to be the rms at the centre of a single pointing (same as ``rmsbeam``), this shows hence the increase in signal-to-noise (sensitivity) relative to the signal-to-noise (sensitivity) at the centre of a single beam. The weight map (cube) can be returned as a fits data cube with the name specified in the parameter ``weightmapname``. Then the noise map, the inverse of the weight map (cube) in units of Jy is calculated, clipped at values above the ``clipper`` parameter. This map (cube), representing the expected rms across the field, can be saved as a fits file with the name ``rmsmapname``.

In the following some statistics is carried out for pixel values in the noise cube below the clip value ``statmax`` and, if a polygon was specified, inside the polygon. If the latter statistics should not be carried out, ``polypointsra`` should be an empty list. A viewgraph is generated showing the area having an rms noise below a threshold ``rms_max``. This, togeter with the numbers is the important figure of merit, which shows you whether your mosaic is wisely chosen. Assuming your science requires an rms of xxx, then you can determine the area that you are covering at this rms and below from this viewgraph. The numbers calculated for the polygon can show you these quantities in your region of interest. The plot can be generated in the format of your choice (e.g. pdf) using the parameter ``rms_statistics_plot`` (e.g. ``rms_statistics_plot = 'rms_statistics.pdf'``).

A map is presented showing the weights cube as a greyscale map with a minimum vmin and maximum vmax for the colour wedge and showing the rms as contours, the levels of which are provided by the user in the list ``rmscontours``. This is done for the channel ``mapchannel`` (index starting with 0) in the rms/weights cube. The plot can be generated in the format of your choice (e.g. pdf) using the parameter ``pointpos_plot`` (e.g. ``pointpos_plot = 'pointpos.pdf'``). If the polygon has been specified (and are not an empty list), the polygon is also overlaid on the mosaic.

As a further step, the mosaic pointings are listed, and can be redirected into a csv file in a format accepted by the MeerKAT observing tool using the parameter ``pointpos_ascii`` (e.g. ``pointpos_ascii = 'pointpos.csv'``)

Finally, a summary PDF file can be generated, containing all (printable) output produced by this notebook. The name of the summary file is specified in the parameter ``summary``, if an empty string '' is provided, no summary file will be generated. WARNING: if ``summary`` is not set to '', the files specified as ``rms_statistics_plot`` and ``pointpos_plot`` will automatically be generated, but with ``.png`` extension. Existing files will be overwritten. If ``summary`` is not '', the default file name for ``rms_statistics_plot`` is ``'rms_statistics_plot.png'`` and the default file name for ``pointpos_plot`` is ``'pointpos_plot.png'``. With the parameter ``summary_th`` the text size in the summary file is specified in pt, with the parameter ``summary_img_width`` the width of figures in the summary file is specified in mm (A4: 210)

In [ ]:
# These should be specified by the user
# For queries contact GIGJ

# Rms in a single pointing
# rmssingle = 0.001     # peak rms for a single beam in Jy, this can also be a list with rmss per beam
rmssingle = 'calculate' # Either a float providing the central rms in a single pointing or 'calculate'

# Parameters to use for the automated calculation of the rms, ignore if rms_single is not 'calculate'
hours = 16               # Integration time in Hours
Nant = 60                # Number of Antennas
velocity_width = 5.513   # in km.s^1, 32k-mode: 5.513 km/s, 4k-mode: 44.108 km/s, n-mode:856000/n
obs_freq =1.421e9        # Observation frequency in Hz
f_tap = 1.               # The weighting factor. Choose 1. for natural weighting, 1.5 for Robust 0

# Determine size of cube
RAmin = 35.         # minimum right ascension of cube in deg
RAmax = 40.         # maximum right ascension of cube in deg
Demin = -16.        # minimum declination of cube in deg
Demax = -13.        # maximum declination of cube in deg
numin = 1300000000. # minimum frequency of cube in Hz
numax = 1400000000. # maximum frequency of cube in Hz
cdeltxy = 0.011     # spatial voxel size in deg
cdeltnu = 10000000  # frequency voxel size in Hz

# Information about pointings and rms, fits file names
ras  = [ 37.,  38.,  37. ,  38. ,  37.,  38.] # Pointings (right ascension, list in deg or hexagesimal,
                                              # e.g ['02h30m0s','02h45m0s'])
decs = [-14., -14., -14.5, -14.5, -15., -15.] # Pointings (declination, list in deg or hexagesimal,
                                              # e.g ['-15d00m0s','-17d30m0s'])
names = ['a','b','c','d','e','f']             # Names of pointings, alternatively empty list for no names

# Or use functionality to create a grid, notice that above values are overridden, polygon is also used
# to calculate statisics
grid_from_polygon = True                                # True for automatically calculating a pointing grid
polypointsra =  [ 35.5, 38., 38., 39.5, 36.5]           # Corners of polygon delimiting grid
                                                        # right ascension in astropy-readable units,
                                                        # empty list [] if no polygon should be specified
polypointsdec = ['-14d00m00s', -13.2, -14., -15., -15.] # Corners of polygon delimiting grid, declination
                                                        # in astropy-readable units
sep = 0.5                                               # Separation of grid points in degree
border = 0.1                                            # Separation from polygon in degree, can also be negative
tilt= 15.                                               # Rotation of grid points in degrees
gridtype = 'hexagonal'                                  # Type of grid, 'quadratic' or 'hexagonal'
startpoint = None                                       # Use None if you want to use the centroid of the polygon
prefix = 'p'                                            # Prefix for automated calculation of pointings

# Output: fits maps
pbshape = 'Tapered Cos Squared' # Primary beam shape
rmsrel = 'rmssingle'            # Either a float or 'rmssingle', if the latter, rmsbeam is used as a reference
weightmapname = 'weights.fits'  # Name of output weights cube (FITS), use empty string if not wanted
clipper = 0.01                  # Clip of noise map in Jy
rmsmapname = 'rms.fits'         # Output FITSfile with rms, use empty string if not wanted

# Output: plotting and statistics
statmax = 0.00075                               # Generate statistics for voxels below this limit
mapchannel = 9                                  # Channel to plot an also be provided as a frequency
                                                # in astropy units
vmin = 0.                                       # Plot lower limit for greyscale
vmax = 1.5                                      # Plot upper limit for greyscale
rmscontours = [0.00005, 0.0001, 0.0002, 0.0004] # contours to plot rms in mosaic
rms_statistics_plot = 'rms_statistics.pdf'      # Statistics plot, set to '' if not required
pointpos_plot = 'pointpos.png'                  # Plot of relative sensitivity, rms, and pointings
pointpos_ascii = 'pointpos.csv'                 # csv file with pointing positions as used by the MeerKAT OPT
summary = 'summary.pdf'                         # Summary file name, empty string '' if not required
                                                # WARNING: if not set to '', rms_statistics_plot and 
                                                # pointpos_plot will automatically be generated, but with
                                                # png extension. Existing files will be overwritten. If summary
                                                # is not '', the default file for rms_statistics_plot that is 
                                                # is rms_statistics_plot.png' and the default file for 
                                                # pointpos_plot is 'pointpos_plot.png'.
summary_th = 11                                 # Summary file text height in pt
summary_img_width = 150                         # SWidth of images in summary file in mm (A4: 210)

In [ ]:
ras, decs, names, hdu, rms, outstring = msc.generate_mos(rmsbeam = rmssingle, t_int = 1., Nant = Nant,\
    velocity_width = velocity_width, obs_freq = obs_freq, f_tap = f_tap, rmsrel = rmsrel, sep = sep, tilt = tilt,\
    grid_from_polygon = grid_from_polygon, polypointsra = polypointsra, polypointsdec = polypointsdec,\
    gridtype = gridtype,     startpoint = startpoint, border = border, prefix = prefix, RAmin = RAmin,\
    RAmax = RAmax, Demin= Demin, Demax = Demax, numin = numin, numax = numax, cdeltxy = cdeltxy,\
    cdeltnu = cdeltnu, ras = ras, decs = decs, names = names, digits = 2, clipper = clipper,\
    rmsmapname = rmsmapname, weightmapname = weightmapname, rms_statistics_plot = rms_statistics_plot,\
    pbshape = pbshape, statmax = statmax, summary = summary)

In [ ]:
outstring = msc.plotmosweight(hdu, channel = mapchannel, rms = rms, vmin=vmin, vmax=vmax, cmap = 'gray_r',\
    rmscontours = rmscontours, rmscol = 'red', ras = ras, decs = decs, names = names, polyra = polypointsra,\
    polydec = polypointsdec, fontsize = 10, thaplot = pointpos_plot, summary = summary, outstring = outstring)

In [ ]:
pointstring  = msc.printmospos(ras = ras, decs = decs, names = names, mosposfile = pointpos_ascii,\
    outstring = '')

In [ ]:
input_dump = msc.gen_input_dump(\
    rmssingle = rmssingle, hours = hours, Nant = Nant, velocity_width = velocity_width, obs_freq = obs_freq,\
    f_tap = f_tap, dummy1 = '', dummy2 = '', RAmin = RAmin, RAmax = RAmax, Demin = Demin, Demax = Demax,\
    numin = numin, numax = numax, cdeltxy = cdeltxy, cdeltnu = cdeltnu, ras = ras, decs = decs, names = names,\
    dummy3 = '', dummy4 = '', grid_from_polygon = grid_from_polygon, polypointsra = polypointsra,\
    polypointsdec = polypointsdec, sep = sep, border = border, tilt = tilt, gridtype = gridtype,\
    startpoint = startpoint, prefix = prefix, dummy5 = '', dummy6 = '', pbshape = pbshape, rmsrel = rmsrel,\
    weightmapname = weightmapname, clipper = clipper, rmsmapname = rmsmapname, statmax = statmax,\
    mapchannel = mapchannel, vmin = vmin, vmax = vmax, rmscontours = rmscontours,\
    rms_statistics_plot = rms_statistics_plot, pointpos_plot = pointpos_plot, pointpos_ascii = pointpos_ascii,\
    summary = summary, summary_th = summary_th, summary_img_width = summary_img_width)

msc.gensummary(summary = summary, summary_th = summary_th, outstring = outstring, pointstring = pointstring,\
    pointpos_plot = pointpos_plot, rms_statistics_plot = rms_statistics_plot,\
    summary_img_width = summary_img_width, input_dump = input_dump)